# Model 1

Use the other bactria orders to predict the Top5 bacteria abundance.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lars, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneOut
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
import scipy.stats as stats


# Load data
file_path = '../../matrix/otu_merged_data.csv'
df = pd.read_csv(file_path)

# Function to map drug concentration
def map_drug_concentration(value):
    if value == 0:
        return "none"
    elif value == 5 or value == 10:
        return "low"
    elif value == 50 or value == 100:
        return "high"
    else:
        return "unknown"


# Create 'Drug Set' column
drug_columns = ['amoxicillin', 'oxytetracycline_dihydrate', 'sulfadiazine', 'trimethoprim', 'tylosin_tartrate', 'ciprofloxacin']
df['Drug Set'] = df[drug_columns].apply(lambda row: '_'.join([map_drug_concentration(x) for x in row]), axis=1)

# Filter data to only include specific bacterial families
bacterial_families = ["o__Bacillales;", "o__Lactobacillales;", "o__Enterobacteriales;", "o__Burkholderiales;",
                      "o__Actinomycetales;", "o__Aeromonadales;", "o__Pseudomonadales;"]
df_filtered = df[['SampleID', 'Group', 'Isolation_source', 'Drug Set'] + bacterial_families]

# Prepare training and test sets
train_df = df_filtered[df_filtered['Group'].isin(['G1', 'G2', 'G3'])]
test_df = df_filtered[df_filtered['Group'] == 'G4']

# Label encode the categorical features
label_encoders = {}
for col in ['Isolation_source', 'Drug Set']:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])
    label_encoders[col] = le

# Define features and target variables
X_train = train_df[['Isolation_source', 'Drug Set']]
y_train = train_df[bacterial_families]
X_test = test_df[['Isolation_source', 'Drug Set']]
y_test = test_df[bacterial_families]

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize models
models = {
    'Least Angle Regression': Lars(),
    'Random Forest': RandomForestRegressor(),
    'Lasso': Lasso(),
    'Elastic-Net': ElasticNet()
}

# Initialize evaluation metrics dictionary
evaluation_metrics = {}

# Train and evaluate models
for family in bacterial_families:
    evaluation_metrics[family] = {}
    for model_name, model in models.items():
        # Use Leave-One-Out Cross-Validation
        loo = LeaveOneOut()
        rmse_scores = np.sqrt(-cross_val_score(model, X_train_scaled, y_train[family], cv=loo, scoring='neg_mean_squared_error'))
        
        rmse_mean = np.mean(rmse_scores)
        rmse_std_error = stats.sem(rmse_scores)
        
        # Fit on the entire training set and evaluate on the test set
        model.fit(X_train_scaled, y_train[family])
        y_pred = model.predict(X_test_scaled)
        test_rmse = np.sqrt(mean_squared_error(y_test[family], y_pred))
        
        # Store evaluation metrics
        evaluation_metrics[family][model_name] = {
            'Train RMSE': rmse_mean,
            'Train Standard Error': rmse_std_error,
            'Test RMSE': test_rmse
        }


C:\Users\DuYih\AppData\Local\Temp\ipykernel_5212\1084890260.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[col] = le.fit_transform(train_df[col])
C:\Users\DuYih\AppData\Local\Temp\ipykernel_5212\1084890260.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[col] = le.transform(test_df[col])
C:\Users\DuYih\AppData\Local\Temp\ipykernel_5212\1084890260.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lars, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneOut
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
import scipy.stats as stats

# Load data
file_path = '../../matrix/otu_merged_data.csv'
df = pd.read_csv(file_path)

# Function to map drug concentration
def map_drug_concentration(value):
    if value == 0:
        return "none"
    elif value == 5 or value == 10:
        return "low"
    elif value == 50 or value == 100:
        return "high"
    else:
        return "unknown"

# Create 'Drug Set' column
drug_columns = ['amoxicillin', 'oxytetracycline_dihydrate', 'sulfadiazine', 'trimethoprim', 'tylosin_tartrate', 'ciprofloxacin']
df['Drug Set'] = df[drug_columns].apply(lambda row: '_'.join([map_drug_concentration(x) for x in row]), axis=1)

# Filter data to only include specific bacterial families
bacterial_families = ["o__Bacillales;", "o__Lactobacillales;", "o__Enterobacteriales;", "o__Burkholderiales;",
                      "o__Actinomycetales;", "o__Aeromonadales;", "o__Pseudomonadales;"]
df_filtered = df[['SampleID', 'Group', 'Isolation_source', 'Drug Set'] + bacterial_families]

# Prepare training and test sets
train_df = df_filtered[df_filtered['Group'].isin(['G1', 'G2', 'G3'])]
test_df = df_filtered[df_filtered['Group'] == 'G4']

# Count the number of samples used for training and testing
total_train_samples = train_df.shape[0]
total_test_samples = test_df.shape[0]
print(f"Total training samples: {total_train_samples}")
print(f"Total testing samples: {total_test_samples}")

# Label encode the categorical features
label_encoders = {}
for col in ['Isolation_source', 'Drug Set']:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])
    label_encoders[col] = le

# Define features and target variables
X_train = train_df[['Isolation_source', 'Drug Set']]
y_train = train_df[bacterial_families]
X_test = test_df[['Isolation_source', 'Drug Set']]
y_test = test_df[bacterial_families]

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# # Initialize models
# models = {
#     'Least Angle Regression': Lars(),
#     'Random Forest': RandomForestRegressor(),
#     'Lasso': Lasso(),
#     'Elastic-Net': ElasticNet()
# }

# # Initialize evaluation metrics dictionary
# evaluation_metrics = {}

# # Train and evaluate models
# for family in bacterial_families:
#     evaluation_metrics[family] = {}
#     for model_name, model in models.items():
#         # Use Leave-One-Out Cross-Validation
#         loo = LeaveOneOut()
#         rmse_scores = np.sqrt(-cross_val_score(model, X_train_scaled, y_train[family], cv=loo, scoring='neg_mean_squared_error'))
        
#         rmse_mean = np.mean(rmse_scores)
#         rmse_std_error = stats.sem(rmse_scores)
        
#         # Fit on the entire training set and evaluate on the test set
#         model.fit(X_train_scaled, y_train[family])
#         y_pred = model.predict(X_test_scaled)
#         test_rmse = np.sqrt(mean_squared_error(y_test[family], y_pred))
        
#         # Store evaluation metrics
#         evaluation_metrics[family][model_name] = {
#             'Train RMSE': rmse_mean,
#             'Train Standard Error': rmse_std_error,
#             'Test RMSE': test_rmse
#         }

# Output the total number of samples involved in training and validation
print(f"Total samples for validation (Leave-One-Out): {X_train.shape[0]}")

Total training samples: 425
Total testing samples: 202
Total samples for validation (Leave-One-Out): 425


C:\Users\DuYih\AppData\Local\Temp\ipykernel_11508\3607495147.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[col] = le.fit_transform(train_df[col])
C:\Users\DuYih\AppData\Local\Temp\ipykernel_11508\3607495147.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[col] = le.transform(test_df[col])
C:\Users\DuYih\AppData\Local\Temp\ipykernel_11508\3607495147.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [2]:
evaluation_metrics

{'o__Bacillales;': {'Least Angle Regression': {'Train RMSE': np.float64(0.0038349319622816213),
   'Train Standard Error': np.float64(0.0002299360030446969),
   'Test RMSE': np.float64(0.005937985356894864)},
  'Random Forest': {'Train RMSE': np.float64(0.003927711597789238),
   'Train Standard Error': np.float64(0.0002324900662284059),
   'Test RMSE': np.float64(0.005965682650459183)},
  'Lasso': {'Train RMSE': np.float64(0.004374269283673696),
   'Train Standard Error': np.float64(0.00024933531183090387),
   'Test RMSE': np.float64(0.006586968278482531)},
  'Elastic-Net': {'Train RMSE': np.float64(0.004374269283673696),
   'Train Standard Error': np.float64(0.00024933531183090387),
   'Test RMSE': np.float64(0.006586968278482531)}},
 'o__Lactobacillales;': {'Least Angle Regression': {'Train RMSE': np.float64(4.16189085578734e-05),
   'Train Standard Error': np.float64(5.361665987830978e-06),
   'Test RMSE': np.float64(0.00013801865616042342)},
  'Random Forest': {'Train RMSE': np.flo

In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lars, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
import scipy.stats as stats

# 如果尚未安装 xgboost 或 lightgbm，需要先执行:
# pip install xgboost lightgbm
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

import warnings

warnings.filterwarnings('ignore')

# ============ 1. 读取数据 ============
file_path = '../../matrix/otu_merged_data.csv'
df = pd.read_csv(file_path)

# ============ 2. 构造 Drug Set 列 ============
def map_drug_concentration(value):
    if value == 0:
        return "none"
    elif value == 5 or value == 10:
        return "low"
    elif value == 50 or value == 100:
        return "high"
    else:
        return "unknown"

drug_columns = [
    'amoxicillin', 
    'oxytetracycline_dihydrate', 
    'sulfadiazine', 
    'trimethoprim', 
    'tylosin_tartrate', 
    'ciprofloxacin'
]
df['Drug Set'] = df[drug_columns].apply(
    lambda row: '_'.join([map_drug_concentration(x) for x in row]), axis=1
)

# ============ 3. 过滤数据，保留指定菌科 ============
bacterial_families = [
    "o__Bacillales;", "o__Lactobacillales;", "o__Enterobacteriales;", 
    "o__Burkholderiales;", "o__Actinomycetales;", "o__Aeromonadales;", 
    "o__Pseudomonadales;"
]

df_filtered = df[
    ['SampleID', 'Group', 'Isolation_source', 'Drug Set'] + bacterial_families
]

# ============ 4. 划分训练集和测试集 ============
train_df = df_filtered[df_filtered['Group'].isin(['G1', 'G2', 'G3'])]
test_df = df_filtered[df_filtered['Group'] == 'G4']

# ============ 5. 对类别型变量做标签编码 ============
label_encoders = {}
for col in ['Isolation_source', 'Drug Set']:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])
    label_encoders[col] = le

# ============ 6. 定义特征和目标 ============
X_train = train_df[['Isolation_source', 'Drug Set']]
y_train = train_df[bacterial_families]
X_test = test_df[['Isolation_source', 'Drug Set']]
y_test = test_df[bacterial_families]

# ============ 7. 标准化特征 ============
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ============ 8. 定义模型和超参数网格 ============
model_params = {
    # Lars
    'Least Angle Regression': (
        Lars(),
        {
            'n_nonzero_coefs': [5, 10, 15, None]
        }
    ),
    # Lasso
    'Lasso': (
        Lasso(max_iter=10000),
        {
            'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10]
        }
    ),
    # ElasticNet
    'Elastic-Net': (
        ElasticNet(max_iter=10000),
        {
            'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10],
            'l1_ratio': [0.1, 0.5, 0.9]
        }
    ),
    # Random Forest
    'Random Forest': (
        RandomForestRegressor(random_state=42),
        {
            'n_estimators': [50, 100],
            'max_depth': [None, 5, 10],
            'min_samples_split': [2, 5],
            'min_samples_leaf': [1, 2]
        }
    ),
    # XGBoost
    'XGBoost': (
        XGBRegressor(objective='reg:squarederror', random_state=42),
        {
            'n_estimators': [50, 100],
            'max_depth': [3, 5],
            'learning_rate': [0.01, 0.1],
            'subsample': [0.8, 1.0]
        }
    ),
    # LightGBM
    'LightGBM': (
        LGBMRegressor(random_state=42),
        {
            'n_estimators': [50, 100],
            'max_depth': [3, 5],
            'learning_rate': [0.01, 0.1],
            'subsample': [0.8, 1.0]
        }
    )
}

# ============ 9. 进行训练和评价 ============
results = []  # 用于存储所有结果，最后转换为 DataFrame

loo = LeaveOneOut()

for family in bacterial_families:
    y_train_family = y_train[family]
    y_test_family = y_test[family]
    
    for model_name, (model, param_grid) in model_params.items():
        # --------- 9.1 使用GridSearchCV + LOO寻找最优参数 ---------
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            scoring='neg_mean_squared_error',
            cv=loo,  # LOO 交叉验证
            n_jobs=-1  # 并行
        )
        grid_search.fit(X_train_scaled, y_train_family)
        
        # 最优模型和最优参数
        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_
        
        # --------- 9.2 使用最优模型，再次进行 LOO 来估计训练集性能 ---------
        # （也可以直接从 grid_search.cv_results_ 中获取平均得分，但这里演示手动计算）
        rmse_scores = []
        for train_idx, valid_idx in loo.split(X_train_scaled):
            X_tr, X_val = X_train_scaled[train_idx], X_train_scaled[valid_idx]
            y_tr, y_val = y_train_family.iloc[train_idx], y_train_family.iloc[valid_idx]
            
            best_model.fit(X_tr, y_tr)
            y_val_pred = best_model.predict(X_val)
            rmse_scores.append(
                np.sqrt(mean_squared_error(y_val, y_val_pred))
            )
        
        rmse_mean = np.mean(rmse_scores)
        rmse_sem = stats.sem(rmse_scores) if len(rmse_scores) > 1 else 0
        
        # --------- 9.3 用最优模型在测试集上进行预测并计算 RMSE ---------
        best_model.fit(X_train_scaled, y_train_family)
        y_pred_test = best_model.predict(X_test_scaled)
        test_rmse = np.sqrt(mean_squared_error(y_test_family, y_pred_test))
        
        # --------- 9.4 将结果存入列表 ---------
        results.append({
            'Bacterial Family': family,
            'Model': model_name,
            'Best Params': best_params,
            'Train RMSE Mean': rmse_mean,
            'Train RMSE StdErr': rmse_sem,
            'Test RMSE': test_rmse
        })

# ============ 10. 汇总输出结果为表格 ============
results_df = pd.DataFrame(results)
print("===== 训练与测试结果（部分展示） =====")
print(results_df.head(20))  # 可以自行调整显示数量

# 如果希望完整展示，可用以下方式：
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)
# print(results_df)


425 202
